In [ ]:
import dicom as dm
import pylab
import cv2
import dicom
import pylab
import numpy as np
import pylab as pl
import scipy.misc 
import numpy as np
import matplotlib.pyplot as plt
import glob
from skimage import data, color
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter
from skimage.util import img_as_ubyte
from skimage.util import crop, pad     


In [ ]:
n=213

In [ ]:
def Patient_slice_get_sax_slice(path,patient):
    fullpath=str(path)+str(patient)+'/study/sax_*/'
    L=glob.glob(fullpath)
    F=[]
    for i in L:
        F.append(i.split('_')[1].split('/')[0])
    return F

In [ ]:
def Get_images_path_patient_sax_slice(path,patient,saxslice):
        fullpath=str(path)+str(patient)+'/study/sax_'+saxslice+'/*'
        L=glob.glob(fullpath)
        return L

In [ ]:
def fourier_transform(L):
    ds = dm.read_file(L[0])
    finalArray = np.zeros((ds.pixel_array.shape[0], ds.pixel_array.shape[1], len(L)))
    for index, image in enumerate(L):
        ds = dm.read_file(L[index])
        toFill = ds.pixel_array
        finalArray[:, :, index] = toFill
    ff=np.fft.fft(finalArray)
    return ff

In [ ]:
def harmonic_number(L,n):
    B=fourier_transform(L)
    harmonic_n = np.abs(B[:,:,n])
    pylab.imshow(harmonic_n)
    pylab.show() 
    return  harmonic_n  
    

In [ ]:
def crop_image(  image,  crop_mask  ):
    
    ''' Crop the non_zeros pixels of an image  to a new image 
         
         
    '''
    pxlst = np.where(crop_mask.ravel())[0]
    dims = crop_mask.shape
    imgwidthy = dims[1]   #dimension in y, but in plot being x
    imgwidthx = dims[0]   #dimension in x, but in plot being y
    #x and y are flipped???
    #matrix notation!!!
    pixely = pxlst%imgwidthy
    pixelx = pxlst//imgwidthy

    minpixelx = np.min(pixelx)
    minpixely = np.min(pixely)
    maxpixelx = np.max(pixelx)
    maxpixely = np.max(pixely) 
    crops = crop_mask*image
    img_crop = crop(  crops, ((minpixelx, imgwidthx -  maxpixelx -1 ),
                                (minpixely, imgwidthy -  maxpixely -1 )) )
    return img_crop 

In [ ]:
def high_variance(L):
    x=0
    y=0
    for i in L:
        ds = dm.read_file(i)
        x=ds.pixel_array+x
    x=x/(len(L)) 
    for j in L:
        ds = dm.read_file(j)
        y=(ds.pixel_array-x)**2+y
        y=y/(len(L))
    return y   

In [ ]:
G=Patient_slice_get_sax_slice('/Users/egouda/Downloads/train/',n)

In [ ]:
 D=[]
E=[]
for i in G:
    print i
    LL=Get_images_path_patient_sax_slice('/Users/egouda/Downloads/train/',n,i)
    A=harmonic_number(LL,1)
    B=high_variance(LL)
    D.append(A)
    E.append(B)
    
    

In [ ]:
F=[]

In [ ]:
for i in range(len(D)):
    x=D[i]+E[i]
    F.append(x)          
               

In [ ]:
for t in F:
    pylab.imshow(t)
    pylab.show()
    

In [ ]:
Total_final_array=np.zeros((F[1].shape[0],F[1].shape[1]))
Total_final_array_Fourier = np.zeros((D[1].shape[0],D[1].shape[1]))
Total_final_array_Varience=np.zeros((E[1].shape[0],E[1].shape[1]))

In [ ]:
print Total_final_array_Fourier.shape
print Total_final_array_Varience.shape
print Total_final_array.shape

In [ ]:
kernal=np.ones((5,5),'uint8')
for i in F:
    result = cv2.GaussianBlur(i, (5, 5), 0)
    erode=cv2.erode(result,kernal,iterations=5)
    dilate=cv2.dilate(erode,kernal,iterations=5)
    Total_final_array=Total_final_array+dilate
    

In [ ]:
Total_final_array_Varience=np.zeros((E[1].shape[0],E[1].shape[1]))
for i in E:
    #result = cv2.GaussianBlur(i, (5, 5), 0)
    #erode=cv2.erode(result,kernal,iterations=5)
    #dilate=cv2.dilate(erode,kernal,iterations=5)
    Total_final_array_Varience=Total_final_array_Varience+i

In [ ]:
Total_final_array_Fourier=np.zeros((E[1].shape[0],E[1].shape[1]))
for i in D:
    #result = cv2.GaussianBlur(i, (5, 5), 0)
    #erode=cv2.erode(result,kernal,iterations=5)
    #dilate=cv2.dilate(erode,kernal,iterations=5)
    Total_final_array_Fourier=Total_final_array_Fourier+i

In [ ]:
pylab.imshow(Total_final_array_Fourier)
pylab.show() 

In [ ]:
pylab.imshow(Total_final_array_Varience)
pylab.show() 

In [ ]:
Total_final_array=Total_final_array_Varience+Total_final_array_Fourier

In [ ]:
pylab.imshow(Total_final_array)
pylab.show()

In [ ]:
for j in E:
    pylab.imshow(j)
    pylab.show() 
    

In [ ]:
# Load picture and detect edges
image = Total_final_array
edges = canny(image, sigma=3, low_threshold=10, high_threshold=100)


# Detect two radii
hough_radii = np.arange(20, 35, 2)
hough_res = hough_circle(edges, hough_radii)

# Select the most prominent 5 circles
accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii,
                                           total_num_peaks=5)

# Draw them
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4))
image = color.gray2rgb(image)
for center_y, center_x, radius in zip(cy, cx, radii):
    circy, circx = circle_perimeter(center_y, center_x, radius)
    image[circy, circx] = (220, 20, 20)
#ax.imshow(image)
#plt.show()

In [ ]:
image.shape

In [ ]:
imagex=image[:,:,1]-image[:,:,0]

In [ ]:
pylab.imshow(imagex)
pylab.show()

In [ ]:
edges = canny(imagex, sigma=3, low_threshold=10, high_threshold=50)

In [ ]:
pylab.imshow(edges)
pylab.show()

In [ ]:
height=Total_final_array.shape[0]
width=Total_final_array.shape[1]
binary = np.zeros([height,width],'uint8')


for row in range(0,height):
    for col in range(0, width):
        if Total_final_array[row][col]*edges.sum(axis=0)[col]*edges.sum(axis=1)[row]>0 :
            binary[row][col]=1

In [ ]:
pylab.imshow(binary)
pylab.show()

In [ ]:
binary.shape

In [ ]:
X=[]
for i in G:
    LL=Get_images_path_patient_sax_slice('/Users/egouda/Downloads/train/',n,i)
    for j in LL:
        ds = dm.read_file(j)
        t=ds.pixel_array
        c=crop_image(t,binary)
        X.append(c)

In [ ]:
for i in X:
    pylab.imshow(i,cmap=pylab.cm.bone)
    pylab.show()

In [ ]:
X[1].shape

In [ ]:
len(X)

In [ ]:
TempArray = np.zeros((X[1].shape[0], X[1].shape[1], len(X)))
for index, image in enumerate(X):
        toFill = X[index]
        TempArray[:, :,index] = toFill

In [ ]:
ff_t=np.fft.fft(TempArray)

In [ ]:
harmonic_n_t = np.abs(ff_t[:,:,1])

In [ ]:
pylab.imshow( harmonic_n_t)
pylab.show()

In [ ]:
image = harmonic_n_t
edges = canny(image, sigma=3, low_threshold=10, high_threshold=100)


# Detect two radii
hough_radii = np.arange(20, 35, 2)
hough_res = hough_circle(edges, hough_radii)

# Select the most prominent 5 circles
accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii,
                                           total_num_peaks=2)

# Draw them
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4))
image = color.gray2rgb(image)
for center_y, center_x, radius in zip(cy, cx, radii):
    circy, circx = circle_perimeter(center_y, center_x, radius)
    image[circy, circx] = (220, 20, 20)
#ax.imshow(image)
#plt.show()

In [ ]:
imagex=image[:,:,1]-image[:,:,0]

In [ ]:
imagex=image[:,:,1]-image[:,:,0]
pylab.imshow(imagex)
pylab.show()

In [ ]:
edges = canny(imagex, sigma=3, low_threshold=10, high_threshold=50)

In [ ]:
pylab.imshow(edges)
pylab.show()

In [ ]:
height=harmonic_n_t.shape[0]
width=harmonic_n_t.shape[1]
binary = np.zeros([height,width],'uint8')

for row in range(0,height):
    for col in range(0, width):
        if harmonic_n_t[row][col]*edges.sum(axis=0)[col]*edges.sum(axis=1)[row]>0 :
            binary[row][col]=1

In [ ]:
pylab.imshow(binary)
pylab.show()

In [ ]:
Z=[]
for i in X:
    c=crop_image(i,binary)
    Z.append(c)

In [ ]:
for i in Z:
    pylab.imshow(i)
    pylab.show()

In [ ]:
Z[1].shape

In [ ]:
binary

In [ ]:
index=np.where( binary > 0)

In [ ]:
index[0][0]